<a href="https://colab.research.google.com/github/shubh70abhir/llm-proj-af1500/blob/main/project_1501_(RAG_YOUTUBE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community langchain_groq youtube_transcript_api faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [2]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
import textwrap

In [3]:
embeddings = HuggingFaceEmbeddings()

/tmp/ipython-input-3-3655315981.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipython-input-3-3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
def create_db_from_youtube_video_url(video_url):
  loader = YoutubeLoader.from_youtube_url(video_url)
  transcript = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 100)
  docs = text_splitter.split_documents(transcript)
  db = FAISS.from_documents(docs, embeddings)
  return db

In [5]:
def get_response_from_query(db, query, k = 4):
  docs = db.similarity_search(query, k = k)
  docs_page_content = " ".join([d.page_content for d in docs])
  chat = ChatGroq(api_key = "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", model = "llama-3.3-70b-versatile", temperature = 0)
  template = """you are a helpful assistant that can answer question about youtube video
 based on the video's transcript:  {docs}

 only use the factual information from the tanscript to answer the question.
 if you feel like you dont have enough information to answer the question, say "i dont know"."""


  system_message_prompt = SystemMessagePromptTemplate.from_template(template)
  human_template = "Answer the following question: {question}"
  human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

  chat_prompt = ChatPromptTemplate.from_messages(
      [system_message_prompt, human_message_prompt]
  )

  chain = LLMChain(llm = chat, prompt = chat_prompt)

  chain.run(question = query, docs = docs_page_content)

  response = chain.run(question = query, docs = docs_page_content)

  response = response.replace("\n","")

  return response, docs

In [6]:
video_url = "https://youtu.be/XjV4HYZTJB8?si=fPIcMM92LJIdQ4b8"
db = create_db_from_youtube_video_url(video_url)
query = "what is this video about"
response, docs = get_response_from_query(db, query)
print(textwrap.fill(response,width = 70))


/tmp/ipython-input-5-3464384132.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = chat, prompt = chat_prompt)
/tmp/ipython-input-5-3464384132.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(question = query, docs = docs_page_content)


This video is about a 7-day dopamine detox reset, which involves
several pillars, including: 1. Limiting mobile screen time to 1 hour a
day, 2. No YouTube, 3. No adult films or self-pleasure with imagery,
4. 10 minutes a day of meditation. The goal of the reset is to help
people derive pleasure from simple things, increase focus, and achieve
their goals.


In [7]:
print(docs)

[Document(id='59be68a2-3eb7-42eb-9908-609e7f8ae878', metadata={'source': 'XjV4HYZTJB8'}, page_content="this is a much longer video that i can talk about but like when you put things on pedestal whether that be a supercar or a potential partner or literally whatever it is to me it's just you're creating a separation from it so unless this is someone who you actually have a chance with like for me just unfollow all these instagram models that you follow and stuff like that and i'll be honest probably a lot of the girls that you guys are following on instagram i know personally in real life we've hung out we've spent time together et cetera et cetera these are people i i literally know in real life and i even mute all their stuff that's a brilliant invention on instagram just mute people's stories mute people's posts because when i go on my feed and i'm really dialed in i don't need to be deriving pleasure from something that's digital like that and not like an actual physical connection 

In [10]:
!pip install translate

In [11]:
from translate import Translator

translator = Translator(to_lang="hi")  # "hi" is the language code for Hindi
translated_response = translator.translate(response)

print("Original Response:")
print(textwrap.fill(response, width=70))
print("\nTranslated Response (Hindi):")
print(textwrap.fill(translated_response, width=70))

Original Response:
This video is about a 7-day dopamine detox reset, which involves
several pillars, including: 1. Limiting mobile screen time to 1 hour a
day, 2. No YouTube, 3. No adult films or self-pleasure with imagery,
4. 10 minutes a day of meditation. The goal of the reset is to help
people derive pleasure from simple things, increase focus, and achieve
their goals.

Translated Response (Hindi):
यह वीडियो 7 - दिवसीय डोपामाइन डिटॉक्स रीसेट के बारे में है, जिसमें कई
स्तंभ शामिल हैं, जिनमें शामिल हैं: 1. मोबाइल स्क्रीन समय को दिन में 1
घंटे तक सीमित करना, 2. कोई यूट्यूब नहीं, 3. कोई वयस्क फिल्में या
इमेजरी के साथ आत्म - आनंद नहीं, ध्यान के दिन 4.10 मिनट। रीसेट का
लक्ष्य लोगों को सरल चीजों से आनंद प्राप्त करने, फोकस बढ़ाने और अपने
लक्ष्यों को प्राप्त करने में मदद करना है।
